## Get Deals and M&A data (part 2)

This notebook will demosntrate how to ingest M&A deals data using both Screener and Search capabilities of the Data Libraries. Additionally, we will compare ingestion of financial ratios for target companies and those for peers. The resulting datastet for target and peer companies can be used for further analyitcs, e.g M&A predictive modeling.

#### Learn more

To learn more about the Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
this [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 

----

Below we import the required libraries.

In [1]:
import refinitiv.data as rd
import pandas as pd
import datetime
import time
from refinitiv.data.discovery import Peers

rd.open_session()

<refinitiv.data.session.Definition object at 0x7f97d0eaa0d0 {name='codebook'}>

### M&A deals Ingestion

There are multiple ways of Ingesting M&A deals data, including via Workspace App and the APIs. One can get M&A data using MASRCH deal screener app inside the Workspace, however, it is  not the most scalable approach of data retrieval for downstream analysis ad we would want to access the data programmatically and integrate it into our analysis pipeline. There are two ways we can get M&A data via the API - using Search or Screener.

Using Screener for M&A deals ingestion. The code below creates a screener with the following filtering criteria:
* M&A Announcement date - between 2023-08-10 and 2023-11-20
* M&A Target Nation - United States and United Kingdom
* M&A Status - Completed, Pending or Withdrawn
* M&A type - Merger or Acquisition
* Target company should be publis
* Acquirers should not be shareholder or creditors

In [3]:
criteria = "SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/), BETWEEN(TR.MnAAnnDate,20230801,20231120)/*dt:Date*/, IN(TR.MnATargetNation,""US"",""GB""), \
            IN(TR.MnAStatus,""W"",""C"",""P""), IN(TR.MnAFormType,""A"",""M""), IN(TR.MnAPubStatus,""P""),NOT_IN(TR.MnAAcquirorCUTIC,""81926M""), \
            NOT_IN(TR.MnAAcquirorCUTIC,""22542L""), CURN=USD)"

display = ["TR.MnAAnnDate",
           "TR.MnAAcquirorRic",
           "TR.MnAAcquirorPermId",
           "TR.MnADealValue",
           "TR.MnAStatus",
           "TR.MnAAcquiror",
           "TR.MnATarget",
           "TR.MnAAcquirorNation",
           "TR.MnATargetNation",
           "TR.MnAPctOfSharesAcquired",
           "TR.MnAPctOfSharesOwnedPostMerger",
           "TR.MnAFormType"
           ]
deals_screener = rd.get_data(criteria, display)
deals_screener

,Instrument,Date Announced,Acquiror PermID,Deal Value,Deal Status,Acquiror Full Name,Target Full Name,Acquiror Nation,Target Nation,Percentage of Shares Acquired in Transaction,Percentage of Shares Owned after Transaction,Form of the Deal
0,154087472867,2023-09-21,5080018615,28133135000,Pending,Cisco Systems Inc,Splunk Inc,United States,United States,0,<NA>,Merger
1,154087939936,2023-08-18,5079222427,200800000,Completed,RI UK 1 Ltd,Ediston Property Investment Co PLC,Jersey,United Kingdom,100,100,Merger
2,154087960871,2023-10-11,4295912121,60273912000,Pending,Exxon Mobil Corp,Pioneer Natural Resources Co,United States,United States,0,<NA>,Merger
3,154088005806,2023-08-09,<NA>,1191220000,Completed,Artisan Bidco Inc,Avid Technology Inc,United States,United States,100,100,Merger
4,154088095821,2023-08-08,4298015188,1666107000,Pending,EchoStar Corp,DISH Network Corp,United States,United States,0,<NA>,Merger
...,...,...,...,...,...,...,...,...,...,...,...,...
99,154088272415,2023-11-13,5073144848,125100000,Pending,Telix Pharmaceuticals (US) Inc,QSAM Biosciences Inc,United States,United States,0,<NA>,Merger
100,154088274447,2023-11-16,4295898916,151342000,Pending,Young & Co's Brewery PLC,The City Pub Group PLC,United Kingdom,United Kingdom,0,<NA>,Merger
101,154088274685,2023-11-16,5086881908,515766000,Pending,Hive Bidco Inc,Hotel Chocolat Group PLC,United States,United Kingdom,0,<NA>,Merger
102,154088276902,2023-11-20,5086736054,19699000,Pending,Rotala Group Ltd,Rotala PLC,United Kingdom,United Kingdom,0,<NA>,Merger


Sometimes screener function can get complicated especially when we provide many filtering critera. And in those cases the Search capabilities of Data Library become more handy. The code below does uses the same filtering criteria to ingest the M&A deals.

In [4]:
deals_search = rd.discovery.search(
    view = rd.discovery.Views.DEALS_MERGERS_AND_ACQUISITIONS,
 
    #specify filtering properties
    filter = "((AcquirerCompanyName ne 'Creditors' and AcquirerCompanyName ne 'Shareholder') and (TargetCountry eq 'US' or TargetCountry eq 'UK')"
    + "and TargetPublicStatus eq 'Public')"
    + "and (TransactionStatus eq 'Completed' or TransactionStatus eq 'Pending' or TransactionStatus eq 'Withdrawn')"
    + "and (FormOfTransactionName xeq 'Merger' or FormOfTransactionName xeq 'Acquisition') and (TransactionAnnouncementDate ge 2023-08-01 and TransactionAnnouncementDate le 2023-11-20)",
    #select only the required fields and order them based on announcement date
    #then specify number of items to be 10000, default value is 100
    select = 'TransactionAnnouncementDate, AcquirerName, AcquirerRIC, AcquirerOAPermId, AcquirerCountry, TargetCompanyName, TargetRIC, TargetCountry, TransactionValueIncludingNetDebtOfTarget, PercentageOfSharesAcquired, DTSimpleType',
    order_by = 'TransactionAnnouncementDate desc',
    top = 10000)
 
deals_search

,TransactionAnnouncementDate,AcquirerName,AcquirerRIC,AcquirerOAPermId,AcquirerCountry,TargetCompanyName,TargetRIC,TargetCountry,TransactionValueIncludingNetDebtOfTarget,PercentageOfSharesAcquired,DTSimpleType
0,2023-11-20,[Rotala Group Ltd],<NA>,[5086736054],UK,Rotala PLC,[ROTL.L],UK,81.608,0,Merger
1,2023-11-20,[Investor Group],<NA>,<NA>,UK,Velocys PLC,[VLSV.L],UK,5.145,0,Merger
2,2023-11-16,[Hive Bidco Inc],<NA>,[5086881908],US,Hotel Chocolat Group PLC,[HOTC.L],UK,640.478,0,Merger
3,2023-11-16,[Young & Co's Brewery PLC],[YNGa.L],[4295898916],UK,The City Pub Group PLC,[CPC.L],UK,215.516,0,Merger
4,2023-11-13,[Telix Pharmaceuticals US Inc],<NA>,[5073144848],US,QSAM Biosciences Inc,[QSAM.PK],US,125.404,0,Merger
...,...,...,...,...,...,...,...,...,...,...,...
99,2023-08-07,[Authentic Restaurant Brands],<NA>,<NA>,US,Fiesta Restaurant Group Inc,[FRGI.O^J23],US,196.916,100,Merger
100,2023-08-04,[Eureka Investor Group Inc],<NA>,<NA>,US,Eureka Homestead Bancorp Inc,[ERKH.PK],US,13.0,0,Merger
101,2023-08-02,[White Bidco Ltd],<NA>,[5086708189],UK,Blancco Technology Group PLC,[F2B.BE],UK,205.569,100,Merger
102,2023-08-01,[Revolution Medicines Inc],[RVMD.O],[5044506598],US,EQRX Inc,[EQRX.O^K23],US,320.661,100,Merger


More on how you can use search, including guidance, examples, and tips to determine the possible approaches, from simple discovery through experimentation to more advanced techniques, are presented in this article.

### Getting financial performance data for target companies

In order to get the financial ratios for the list of companies we use get_data function. We specify the ratios under the field argument and the date in the parameters. Here we will provide several workflows of ingesting the data, including:
* ingestion as of a universal date
* ingestion with different dates for different assets
* ingesting with multiple dates across the assets

Before proceeding with the ingestion let's define our rics, dates and the fields:

In [5]:
deals_search['TargetRIC'] = deals_search['TargetRIC'].explode()
deals_search = deals_search.drop_duplicates(subset='TargetRIC')
deals_search = deals_search.dropna(subset=['TargetRIC'])

In [6]:
target_rics = deals_search['TargetRIC'].to_list()

fields = ["TR.F.MktCap","TR.F.TotRevenue", "TR.F.ReturnAvgTotEqPctTTM", "TR.F.IncAftTaxMargPctTTM", "TR.F.GrossProfMarg", "TR.F.CurrRatio",
           "TR.F.ReturnCapEmployedPctTTM","TR.F.WkgCaptoTotAssets","TR.PriceToBVPerShare","TR.PriceToSalesPerShare","TR.EVToSales",
           "TR.TotalDebtToEV","TR.F.NetDebtPerShr"]

#### Ingesting as of a universal date

Since the request is as of a date for all companies in our list, the request workflow doesn't require loop procedures and multiple API requests. This allows us to receive the data relatively faster. We time the request to compare with the workflow involving looping and multiple API requests.

In [7]:
start_time = time.time()
target_data_as_of = rd.get_data(universe=target_rics, fields = fields , parameters = {'SDate': '2021-12-12'})

print("--- %s seconds ---" % (time.time() - start_time))
target_data_as_of

--- 2.364733934402466 seconds ---


,Instrument,Market Capitalization,Revenue from Business Activities - Total,"Return on Average Total Equity - %, TTM","Income after Tax Margin - %, TTM",Gross Profit Margin - %,Current Ratio,"Return on Capital Employed - %, TTM",Working Capital to Total Assets,Price To Book Value Per Share (Daily Time Series Ratio),Price To Sales Per Share (Daily Time Series Ratio),Enterprise Value To Sales (Daily Time Series Ratio),Total Debt To Enterprise Value (Daily Time Series Ratio),Net Debt per Share
0,ROTL.L,14025510.52,78115000.0,1.806284,0.634031,15.49638,0.73363,-23.208675,-0.09012,0.461948,0.161762,0.88372,81.520673,1.302087
1,VLSV.L,108011839.7855,178000.0,-146.659243,-95.956284,43.25843,2.14375,-133.92945,0.46937,7.435953,10.633486,9.752611,1.36342,-0.011077
2,HOTC.L,462598555.65,164551000.0,5.538197,2.235173,61.78875,1.08474,11.199842,0.02934,10.172443,4.436172,4.619459,5.289313,0.239593
3,CPC.L,85601525.625,25815000.0,-4.534541,-19.099579,75.67306,1.52834,-7.889331,0.03727,1.080897,4.468946,5.974823,32.460452,0.311519
4,QSAM.PK,7917250.2,0.0,<NA>,<NA>,<NA>,0.00506,<NA>,-196.63292,11.087627,<NA>,<NA>,0.479196,8.064833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,FRGI.O^J23,295497439.2,315358000.0,0.82023,0.357206,61.28559,1.29452,0.436962,0.03361,1.497567,0.70701,0.563346,0.429039,0.921514
96,ERKH.PK,15499545.6,2192000.0,0.701412,6.235201,<NA>,<NA>,0.701412,<NA>,0.760611,5.967585,3.118231,0.0,12.792943
97,F2B.BE,202177291.8,36506000.0,1.812796,3.780201,92.31085,2.07997,1.159077,0.10083,2.475693,5.041309,4.829344,1.00397,-0.10983
98,EQRX.O^K23,<NA>,<NA>,<NA>,<NA>,<NA>,0.65986,<NA>,-0.48815,<NA>,<NA>,<NA>,<NA>,0.015184


#### Ingesting with different dates across the assets

Most of the usecases may require ingesting teh company financials as of different dates, for example as of some day before the M&A announcement.
Again, we use the get_data function, however, we call it under a loop by updating the date parameter. Since the process involves multiple API requests, it is advised to use a try - except statement to catch and respond to potential errors.

For our use case we want our feature space to be representative to the company fundamentals landscape 30 days prior to the M&A announcement. The snippet below creates a dataframe of deals containing dates materialised 30 days prior to the Announcement date.

In [8]:
#create list of announcement dates including one for 30 days prior to the announcement
deals_search['AD_30'] = (pd.to_datetime(deals_search['TransactionAnnouncementDate']) - datetime.timedelta(30)).dt.strftime('%Y-%m-%d')
request_dates = deals_search['AD_30'].to_list()

In [9]:
start_time = time.time()

target_data = pd.DataFrame()
retries = 0
for i in range(len(target_rics)):

    while retries < 3:
        try:
            #get data for fields as of the specified date 
            df = rd.get_data(universe=target_rics[i], fields = fields , parameters = {'SDate': request_dates[i]})
            #add anoouncement date to the resulting dataframe
            df['AD_30'] = request_dates[i]

            #append company data to the main dataframe
            target_data = pd.concat([target_data, df], ignore_index = True, axis = 0)
        except Exception as e:
            # print(e)
            retries += 1
            continue
        break

print ("--- %s seconds ---" % (time.time() - start_time))

--- 73.94764971733093 seconds ---


In [10]:
target_data

,Instrument,Market Capitalization,Revenue from Business Activities - Total,"Return on Average Total Equity - %, TTM","Income after Tax Margin - %, TTM",Gross Profit Margin - %,Current Ratio,"Return on Capital Employed - %, TTM",Working Capital to Total Assets,Price To Book Value Per Share (Daily Time Series Ratio),Price To Sales Per Share (Daily Time Series Ratio),Enterprise Value To Sales (Daily Time Series Ratio),Total Debt To Enterprise Value (Daily Time Series Ratio),Net Debt per Share,AD_30
0,ROTL.L,16729224.68,84871000,-3.351771,-0.925719,12.08893,0.58357,1.320558,-0.08904,0.824519,0.170038,0.634846,74.49184,0.77539,2023-10-21
1,VLSV.L,65690538.457,241000,-90.48566,-6807.086614,63.90041,4.35101,-52.172515,0.44615,1.407309,76.737119,59.571764,30.817062,-0.00927,2023-10-21
2,HOTC.L,163669830.87,204500000,-6.495293,-3.046944,58.28949,1.556,-2.101903,0.12664,2.089277,0.955029,1.137784,20.612705,0.267309,2023-10-17
3,CPC.L,75211547.125,57793000,0.063667,0.093183,75.6666,0.56368,2.68598,-0.05075,1.006723,1.477645,1.828409,24.205403,0.21301,2023-10-17
4,QSAM.PK,8169103.5,0,,,,0.23077,,-3.33335,-16.470393,,,0.047947,0.099132,2023-10-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,FRGI.O^J23,191026639.65,387351000,-11.102702,-4.177033,62.66745,1.3074,-2.729452,0.03553,1.44392,0.534983,0.459916,0.223972,-1.254154,2023-07-08
96,ERKH.PK,16623196.65,2500000,0.249275,2.119853,,,0.249275,,0.589484,4.054291,2.703085,0,9.148836,2023-07-05
97,F2B.BE,150589100.85,39799000,3.520149,6.258494,96.75871,1.62641,3.863613,0.07146,1.57201,2.915347,2.748749,0.412025,-0.047137,2023-07-03
98,EQRX.O^K23,1201546097.58,,-19.02736,,,27.50763,-25.047713,0.94581,0.680466,,,0,-2.923253,2023-07-02


#### Optimising ingestion request

One important concern during data ingestion is making sure that we have an optimised ingestion layer that is not presenting any unnecessary bottlenecks within our pipeline. This is a proprietary decision, and, in this case, we did notice that there are dates for which multiple M&A has been announced. To optimise our ingestion request we could bundle the deals by date. That will allow us to reduce API calls and retrieve the results much faster.

The code below groups our dataframe by the AD_30 and makes an API call for each date requesting feature values for the list of rics instead of making individual call for each ric.

In [11]:
start_time = time.time()

dfs = dict(tuple(deals_search.groupby('AD_30', sort = False)))

#create empty lists and dataframe to store requested values
target_data = pd.DataFrame()
retries = 0
for date in dfs:
    rics = []
    for ric in dfs[date]['TargetRIC']:
        rics.append(ric)
        #get data for fields as of the specified date
    while retries < 3:
        try:
            df = rd.get_data(universe = rics, fields = fields , parameters = {'SDate': date})
            #add anoouncement date to the resulting dataframe
            df['AD_30'] = date
            #append company data to the main dataframe
            target_data = pd.concat([target_data, df], ignore_index = True, axis = 0)
        except:
            retries += 1
            continue
        break

print("--- %s seconds ---" % (time.time() - start_time))

--- 37.84473490715027 seconds ---


In [11]:
target_data

,Instrument,Market Capitalization,Revenue from Business Activities - Total,"Return on Average Total Equity - %, TTM","Income after Tax Margin - %, TTM",Gross Profit Margin - %,Current Ratio,"Return on Capital Employed - %, TTM",Working Capital to Total Assets,Price To Book Value Per Share (Daily Time Series Ratio),Price To Sales Per Share (Daily Time Series Ratio),Enterprise Value To Sales (Daily Time Series Ratio),Total Debt To Enterprise Value (Daily Time Series Ratio),Net Debt per Share,AD_30
0,ROTL.L,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,<NA>,0.824519,0.170038,0.634846,74.49184,<NA>,2023-10-21
1,VLSV.L,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,<NA>,1.407309,76.737119,59.571764,30.817062,<NA>,2023-10-21
2,HOTC.L,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,<NA>,2.089277,0.955029,1.137784,20.612705,<NA>,2023-10-17
3,CPC.L,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,<NA>,1.006723,1.477645,1.828409,24.205403,<NA>,2023-10-17
4,QSAM.PK,8169103.5,0,NaN,NaN,NaN,0.23077,<NA>,-3.33335,-16.470393,<NA>,<NA>,0.047947,0.099132,2023-10-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,WRK,7855266795.39,21256500000,-13.115486,-6.803607,18.91516,1.53362,5.935803,0.07332,0.890973,0.430218,0.851843,51.133295,29.587264,2023-08-08
61,TNT.L,23953186.95,0,-904.984424,<NA>,<NA>,0.93729,-347.340426,-0.05809,7.088297,<NA>,<NA>,3.308029,-0.539742,2023-08-08
62,NXGN.O^K23,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2.759966,1.670328,1.742678,22.60834,<NA>,2023-08-07
63,BKCC.O,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.841202,3.659258,7.409997,52.936362,<NA>,2023-08-07


### Ingesting with multiple dates accross the assets

There might be usecases, where we would want to ingest data for an asset with muliple dates, for example, let's assume we want financial ratios as of 30 days prior to the announcement as shown above, but also as of the rest of the requested dates we had for the rest of the assets.

For that we need to create a new list of fields which will include a date of request. The code below does exactly that.

In [12]:
new_fields = []
for i in range(len(target_rics)):
    for field in fields:
        new_fields.append(f'{field}(SDate={request_dates[i]})')
new_fields = list(set(new_fields))
print(new_fields[:20])

['TR.F.TotRevenue(SDate=2023-08-16)', 'TR.TotalDebtToEV(SDate=2023-10-02)', 'TR.F.IncAftTaxMargPctTTM(SDate=2023-07-31)', 'TR.F.NetDebtPerShr(SDate=2023-08-27)', 'TR.F.NetDebtPerShr(SDate=2023-07-19)', 'TR.EVToSales(SDate=2023-10-01)', 'TR.F.IncAftTaxMargPctTTM(SDate=2023-07-10)', 'TR.TotalDebtToEV(SDate=2023-10-01)', 'TR.TotalDebtToEV(SDate=2023-10-07)', 'TR.TotalDebtToEV(SDate=2023-07-09)', 'TR.F.IncAftTaxMargPctTTM(SDate=2023-10-08)', 'TR.EVToSales(SDate=2023-10-17)', 'TR.F.GrossProfMarg(SDate=2023-07-09)', 'TR.F.MktCap(SDate=2023-09-27)', 'TR.EVToSales(SDate=2023-07-09)', 'TR.PriceToBVPerShare(SDate=2023-10-21)', 'TR.F.MktCap(SDate=2023-09-24)', 'TR.F.GrossProfMarg(SDate=2023-08-29)', 'TR.TotalDebtToEV(SDate=2023-07-18)', 'TR.TotalDebtToEV(SDate=2023-10-14)']


Here we use an additional parameter use_field_names_in_headers and set it true to show the field names with a date in the dataframe

In [13]:
start_time = time.time()

target_data_multi_days = rd.get_data(target_rics, new_fields, use_field_names_in_headers = True)

print("--- %s seconds ---" % (time.time() - start_time))
target_data_multi_days

--- 34.457295179367065 seconds ---


,Instrument,TR.F.TOTREVENUE(SDATE=2023-08-16),TR.TOTALDEBTTOEV(SDATE=2023-10-02),TR.F.INCAFTTAXMARGPCTTTM(SDATE=2023-07-31),TR.F.NETDEBTPERSHR(SDATE=2023-08-27),TR.F.NETDEBTPERSHR(SDATE=2023-07-19),TR.EVTOSALES(SDATE=2023-10-01),TR.F.INCAFTTAXMARGPCTTTM(SDATE=2023-07-10),TR.TOTALDEBTTOEV(SDATE=2023-10-01),TR.TOTALDEBTTOEV(SDATE=2023-10-07),...,TR.PRICETOSALESPERSHARE(SDATE=2023-10-14),TR.F.NETDEBTPERSHR(SDATE=2023-09-12),TR.EVTOSALES(SDATE=2023-09-26),TR.PRICETOBVPERSHARE(SDATE=2023-08-07),TR.PRICETOSALESPERSHARE(SDATE=2023-07-21),TR.F.RETURNCAPEMPLOYEDPCTTTM(SDATE=2023-08-29),TR.F.INCAFTTAXMARGPCTTTM(SDATE=2023-10-02),TR.F.MKTCAP(SDATE=2023-09-19),TR.F.GROSSPROFMARG(SDATE=2023-09-11),TR.F.INCAFTTAXMARGPCTTTM(SDATE=2023-08-16)
0,ROTL.L,84871000,72.689145,-0.925719,0.77539,0.77539,0.65059,1.375028,72.689145,72.689145,...,0.185782,0.77539,0.65059,0.679465,0.162655,1.320558,-0.925719,16729224.68,12.08893,-0.925719
1,VLSV.L,241000,35.799877,-5476.763485,-0.00927,-0.00927,51.767989,-5476.763485,35.462586,38.353379,...,60.479254,-0.00927,56.157613,1.407051,111.719185,-51.672507,-6807.086614,65690538.457,63.90041,-5476.763485
2,HOTC.L,226133000,24.995325,-9.11925,0.317698,0.317698,0.938626,-9.11925,25.68317,25.68317,...,0.958392,0.317698,0.948312,1.464923,0.768443,12.030428,-9.11925,391357388.7,58.51556,-9.11925
3,CPC.L,57793000,24.301107,1.645528,0.21301,0.21301,1.812947,1.645528,24.411839,24.205403,...,1.477645,0.21301,1.788535,1.004037,1.709148,2.831669,0.093183,75211547.125,75.6666,1.645528
4,QSAM.PK,0,0.045553,<NA>,0.099132,0.099132,<NA>,<NA>,0.045282,0.046105,...,<NA>,0.099132,<NA>,-1962.218641,<NA>,<NA>,<NA>,8169103.5,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,FRGI.O^J23,387351000,0.206731,-4.177033,-1.254154,-1.254154,0.463976,-4.177033,0.206731,0.206442,...,0.549786,-1.254154,0.462678,1.491991,0.526408,-0.207717,-1.533451,191026639.65,62.66745,-1.533451
96,ERKH.PK,2500000,0.0,2.119853,9.148836,9.148836,5.147476,2.119853,0.0,0.0,...,<NA>,9.148836,5.35056,0.966367,4.682004,0.249275,2.119853,16623196.65,<NA>,2.119853
97,F2B.BE,39799000,0.311098,6.258494,-0.047137,-0.047137,3.657651,6.258494,0.30964,0.320147,...,3.772803,-0.047137,3.6748,2.05286,3.129711,3.863613,6.258494,150589100.85,96.75871,6.258494
98,EQRX.O^K23,<NA>,0.0,<NA>,-2.923253,-2.923253,<NA>,<NA>,0.0,0.0,...,<NA>,-2.923253,<NA>,0.812169,<NA>,-25.242645,<NA>,1201546097.58,<NA>,<NA>


### Getting financial performance data for non-target companies (peers)

The non-target sample set is constructed from peer companies. The code below gets the company peers and requests the fields we have defined above.

In [14]:
#create empty lists for error data and a dataframe to store selected peers
retries = 0
peer_data = pd.DataFrame()
for i in range(len(target_rics)):
    while retries < 3:
        try:
            #request Peer function for each target company in the lits
            instruments =list(Peers(target_rics[i]))
            vals = rd.get_data(universe=instruments, fields = fields, parameters = {'SDate': request_dates[i]})

            #add a column for 30 days prior to the M&A announcement
            vals['AD_30'] = request_dates[i]

            #append target company's peer data to the main dataframe of all peers
            peer_data = pd.concat([peer_data, vals], ignore_index = True, axis = 0)

        #if error is returned, store ric and request date in a separate list    
        except:
            retries += 1
            continue
peer_data

,Instrument,Market Capitalization,Revenue from Business Activities - Total,"Return on Average Total Equity - %, TTM","Income after Tax Margin - %, TTM",Gross Profit Margin - %,Current Ratio,"Return on Capital Employed - %, TTM",Working Capital to Total Assets,Price To Book Value Per Share (Daily Time Series Ratio),Price To Sales Per Share (Daily Time Series Ratio),Enterprise Value To Sales (Daily Time Series Ratio),Total Debt To Enterprise Value (Daily Time Series Ratio),Net Debt per Share,AD_30
0,ALQ.MC,42110872.5,100364652.95,25.527191,8.485269,50.81588,0.2813,13.855781,-0.29506,1.413561,0.519343,2.036779,78.026474,27.760788,2023-10-21
1,SBC.MI,<NA>,152361978.0,<NA>,<NA>,87.61381,1.01509,<NA>,0.00499,2.656492,1.34417,1.462917,21.979232,0.569482,2023-10-21
2,HVBG.H,316800000.0,164037000.0,-14.743266,-16.12563,42.88728,3.10837,-6.259694,0.18004,1.113867,1.850802,1.727781,34.68492,-0.764394,2023-10-21
3,GREENM.CO,191308462.0,97310000.0,<NA>,<NA>,100.0,0.76727,<NA>,-0.0722,4.017564,1.632212,3.338511,55.479643,35.475117,2023-10-21
4,DONKEY.CO,17806632.437143,9084061.0,-55.916636,-35.730523,-0.53111,1.06968,-22.981085,0.02095,2.318334,1.424675,2.072408,44.762202,0.423576,2023-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8165,RAILG.ST,481518373.32,497035000.0,16.912827,7.514787,39.72577,0.94261,14.284119,-0.01457,1.733079,0.812163,1.177893,33.82531,7.713759,2023-10-21
8166,ANKr.AT,46519133.01,180012000.0,<NA>,-7.581231,5.85572,0.13754,<NA>,-0.99793,-0.510774,0.292725,1.755139,84.4618,1.42432,2023-10-21
8167,ALTPC.PA,158196250.0,35019785.0,19.155473,38.083419,64.50208,6.69624,13.06089,0.30225,2.273745,4.466026,4.935464,32.124762,2.206565,2023-10-21
8168,EWIND.OL,169615618.056049,26930000.0,-0.019449,-0.161818,100.0,2.53124,0.536853,0.09352,0.789499,8.018578,12.077571,54.127074,1.740556,2023-10-21


In [15]:
rd.close_session()